In [1]:
# importa os pacotes necessários
import numpy as np
import os, cv2, random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
import os
import np_utils
from sklearn import tree

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.neural_network import MLPClassifier

from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
import sklearn.metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import  cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from skimage.feature import greycomatrix, greycoprops
from skimage.feature import hog

In [2]:
#funções de leitura e preparação das imagens
def read_image(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_COLOR) #cv2.IMREAD_GRAYSCALE
    return cv2.resize(img, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)


def prep_data(images):
    count = len(images)
    data = np.ndarray((count, CHANNELS, ROWS, COLS), dtype=np.uint8)

    for i, image_file in enumerate(images):
        image = read_image(image_file)
        data[i] = image.T
        if i%250 == 0: print('Processed {} of {}'.format(i, count))    
    return data
    
# dois exemplos de descritores. Você deve criar outros mais robustos.
def image_to_feature_vector(image, size=(32, 32)):
    # resize the image to a fixed size, then flatten the image into
    # a list of raw pixel intensities
    return cv2.resize(image, size).flatten()

def extract_color_histogram(image, bins=(8, 8, 8)):     
    # extract a 3D color histogram from the HSV color space using
    # the supplied number of `bins` per channel
    #image = cv2.imread(image_file)        
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, bins,
        [0, 180, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    # return the flattened histogram as the feature vector
    return hist.flatten()

In [3]:
TRAIN_DIR = 'kaggle/cifar-10/train'
TRAIN_LABELS_DIR = 'kaggle/cifar-10/trainLabels.csv'

ROWS = 128
COLS = 128
CHANNELS = 3
NIM = 10000

image_paths = os.listdir(TRAIN_DIR)

train_images = [read_image(TRAIN_DIR + "/" + i) for i in image_paths][:NIM]
train_labels = pd.read_csv(TRAIN_LABELS_DIR)

train_labels = train_labels[:NIM]

del train_labels['id']

In [4]:
rawImages = []
descHist = []

count = len(train_images)

for i, image in enumerate(train_images):
    pixels = image_to_feature_vector(image)
    histogram = extract_color_histogram(image)
    
    rawImages.append(pixels)
    descHist.append(histogram)
        
    if i%1000 == 0: print('Processed {} of {}'.format(i, count))
        

Processed 0 of 1000


In [7]:
from keras import applications
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import tensorflow as tf
from keras import utils

def create_model(img_height, img_width, num_classes):
    # Generating model
    base_model = applications.resnet_v2.ResNet50V2(weights=None, include_top=False, input_shape=(img_height, img_width, 1))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.7)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    from keras.optimizers import SGD, Adam

    adam = Adam(lr=1e-4)
    model.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    reduce_on_plateau = ReduceLROnPlateau(monitor="val_accuracy", mode="max", factor=0.1, patience=20, verbose=1)
    earlyStop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=10, verbose=1)

    callbacks_list = [reduce_on_plateau, earlyStop]
    callbacks = callbacks_list
    return callbacks, model

Using TensorFlow backend.
C:\Users\lucas\Anaconda3\envs\reconhecimentopadroes\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\lucas\Anaconda3\envs\reconhecimentopadroes\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\lucas\Anaconda3\envs\reconhecimentopadroes\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])

In [8]:
unique_labels = list(train_labels.label.unique())

train_data = np.array(train_images)
train_images_gray = []
labels_binary = []
labels = np.array(train_labels)

for i in range(len(train_data)):
    input_image = cv2.cvtColor(train_data[i], cv2.COLOR_BGR2GRAY)
    input_image = np.reshape(input_image, (128, 128, 1))
    train_images_gray.append(input_image)
    
    label = labels[i]
    index = unique_labels.index(label)
    
    label = index # [utils.to_categorical(index, num_classes = 10)]
    
    labels_binary.append(label)


In [10]:
num_classes = 1

# Image default sizes
img_height, img_width = 128, 128

print("X_train size: " + str(len(rawImages)))

batch_size = 32
epochs = 5

callbacks, model = create_model(img_height, img_width, num_classes)
train_images_gray = np.array(train_images_gray)
print(train_images_gray.shape)

labels_binary = np.array(labels_binary)
print(labels_binary.shape)

model.fit(train_images_gray, labels_binary,
          batch_size=batch_size,
          epochs=epochs,
          shuffle=True)

preds = model.evaluate(X_test, y_test)

model.save("checkpoint/model_trained_07-10.h5")

print("Loss = " + str(preds[0]))
print("Test Accuracy = " + str(preds[1]))

model.summary()

X_train size: 1000
(1000, 128, 128, 1)
(1000,)
Epoch 1/5
1000/1000 [==============================] - 18s 18ms/step - loss: nan - accuracy: 0.0050
Epoch 2/5
1000/1000 [==============================] - 10s 10ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 3/5
1000/1000 [==============================] - 10s 10ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 4/5
1000/1000 [==============================] - 10s 10ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 5/5
1000/1000 [==============================] - 10s 10ms/step - loss: nan - accuracy: 0.0000e+00


ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 300 arrays: [array([[ 79],
       [ 66],
       [ 44],
       ...,
       [187],
       [196],
       [202]], dtype=uint8), array([[ 69],
       [144],
       [ 85],
       ...,
       [221],
       [234],
      ...